# Caesar Cipher Decryption


The following is an algorithm which utilizes Spark to solve a Caesar Cipher.




In [1]:
import java.io.File
import spark.{sparkContext => sc}
import scala.util.control.Breaks._

In [2]:
def getListOfFiles(dir: String):List[File] = {
    val d = new File(dir)
    if (d.exists && d.isDirectory) {
        d.listFiles.filter(_.isFile).toList
    } else {
        List[File]()
    }
}

# Polynote Multilingual Support

Using Polynote, I can quickly create Python classes which I can call in my Scala cell(s). This comes in very handy for using Python specific packages and for quickly writing easy to read code when performance is negligible.



In [4]:
from scipy.stats import norm
from string import ascii_lowercase
from langdetect import detect_langs

class letterProcessing:
    def alpha2num(self, alpha:str)->int:
        """
        Returns the integer index of the letter in the alphabet
        
        Inputs
        -------
        alpha: string     
            A lowercase letter
        """
       
        try:
            num = ascii_lowercase.index(alpha)
        except ValueError:
            print('Index not found')
            print('Please use a valid lowercase letter')
            num = None
        
        return num

    def num2alpha(self, num:int)->str:
        """
        Returns the letter in the alphabet corresponding to the index
        
        Inputs
        -------
        num: integer    
            The index of a lowercase letter
        """
       
        try:
            alpha = ascii_lowercase[num]
        except IndexError:
            print('Index not found')
            print('Please use a valid index')
            alpha = None
        
        return alpha

    def sampleSize(self, population_size, margin_error=.05, confidence_level=.99, sigma=0.5):
        """
        Calculate the minimal sample size to use to achieve a certain
        margin of error and confidence level for a sample estimate
        of the population mean.

        Inputs
        -------
        population_size: integer
            Total size of the population that the sample is to be drawn from.
        margin_error: number
            Maximum expected difference between the true population parameter,
            such as the mean, and the sample estimate.
        confidence_level: number in the interval (0, 1)
            If we were to draw a large number of equal-size samples
            from the population, the true population parameter
            should lie within this percentage
            of the intervals (sample_parameter - e, sample_parameter + e)
            where e is the margin_error.
        sigma: number
            The standard deviation of the population.  For the case
            of estimating a parameter in the interval [0, 1], sigma=1/2
            should be sufficient.
        """
        
        alpha = 1 - (confidence_level)
        
        # dictionary of confidence levels and corresponding z-scores
        # computed via norm.ppf(1 - (alpha/2)), where norm is
        # a normal distribution object in scipy.stats.
        # Here, ppf is the percentile point function.
        
        zdict = {
            .90: 1.645,
            .91: 1.695,
            .99: 2.576,
            .97: 2.17,
            .94: 1.881,
            .93: 1.812,
            .95: 1.96,
            .98: 2.326,
            .96: 2.054,
            .92: 1.751
        }

        if confidence_level in zdict:
            z = zdict[confidence_level]
        else:
            z = norm.ppf(1 - (alpha/2))
        
        N = population_size
        M = margin_error
        numerator = z**2 * sigma**2 * (N / (N-1))
        denom = M**2 + ((z**2 * sigma**2)/(N-1))

        return int(numerator//denom)

    def langDetect(self, content:str)->float:
        """
        Returns probaility if the word is english, returns the negative 
        probaility of most likely language otherwise

        Inputs
        -------
        content: string 
            The text which you want to classify the language of
        """

        langs = detect_langs(content)
        
        if  langs[0] == 'en':
            return lang.prob > 0.9
        else:
            return False

pyInst = letterProcessing()

In [5]:
import os
os.getcwd()

/home/waldenr1_gmail_com/polynote

In [6]:
// Obtain all files from data folder
val files = getListOfFiles("notebooks/midterm/data")
var succesful = 0
var failed = 0

// Iterate through each file
for (file <- files){

    val filename = file.toString
    println("Processing " + filename + " ...")

    // Extract lines from the encrypted document
    val lines = sc.textFile(filename)

    // Extract words from the encrypted document
    val words = lines
                .flatMap(_.split(" "))
                .filter(_ != "")

    println("Word count: " + words.count)

    // Cleans words of any non alphabetical characters
    val cleanWords = words.map(_.replaceAll("[^a-zA-Z]",""))
    println("Clean word count: " + cleanWords.count);

    // Extract characters from the lines
    val chars = lines
                .flatMap(_.split(""))
                .filter(_ != "")
                .filter(_ != " ")

    println("Character count: " + chars.count)

    // Filters out non alphabetical characters
    val alphaChars = chars
                    .map(_.replaceAll("[^a-zA-Z]",""))
                    .filter(_ != "")

    // Counts the number of total alphabetical characteres
    val total:Int = alphaChars.count.toInt
    println("Alphabetical character count: " + total)

    // Use python class to calculate ideal sample size
    val sample:Int = Integer.parseInt(pyInst.sampleSize(total).toString)
    println("Ideal sample size: " + sample)

    // Crates an integer value (1) for each alphabetical characters
    val lowChars = alphaChars.map(ch => (ch.toLowerCase, 1))

    val e_idx:Int = 4 // The index of 'e', the most common letter, in the alphabet
    val num_let:Int = 26 // The number of letters in the alphabet

    // Takes a statistically appropriate sample of the Words to test
    val bagOfWords = cleanWords.sample(false, sample / cleanWords.count())

    // 
    var charFreq = lowChars.reduceByKey(_ + _) // get unique letter frequencies by reducing key
    var isEnglish:Boolean = false // used for determining succesful decryption
    var iter:Int = 0 // used for counting number of decryption attempts
    var decWords = null // temporary place holder

    // Returns letter with greatest frequency
    var candidate = charFreq.max()(new Ordering[Tuple2[String, Int]]() {
        override def compare(x: (String, Int), y: (String, Int)): Int = Ordering[Int].compare(x._2, y._2)
        })

    // Calculates key using Python function
    var key:Int = Integer.parseInt(pyInst.alpha2num(candidate._1).toString) - e_idx
    
    breakable{
        while (isEnglish == false && iter < num_let){
            println("Candidate e: " + candidate._1)
            println("Candidate's frequency: " + candidate._2)
            
            // Correct negative keys 
            if (key<0){key += num_let}
            println("Most likely key: " + key)

            // Only shift the words if the key value is not zero
            // Need to debug this!!!
            if (key != 0) {
                decWords = bagOfWords
                               .map(word => (for (c <- word) yield pyInst
                               .num2alpha(
                                   (Integer.parseInt(pyInst.alpha2num(c).toString) + e_idx) % 26)
                               .toString))
                               .collect()
                               .toList
            } else {
                decWords = bagOfWords.collect().toList
            }

            isEnglish = (pyInst.langDetect(decWords).toString).toBoolean
            
            if (isEnglish){
                // decrypt all data and write to file
                // Need to implement this!!!
                break
            }
            
            // Adjust relevant variables for next decryption attempt
            iter += 1
            charFreq = charFreq.filter(_ != candidate)
            candidate = charFreq.max()
            key = Integer.parseInt(pyInst.alpha2num(candidate._1).toString) - e_idx
        }
    }

    // handle final results for the current file
    if (isEnglish){
        println(filename + " decryption complete in " + iter + " iterations.")
        println("Caesar Cipher Key: " + key)
        succesful += 1
    }else{
        println(filename + " decryption failed.")
        failed += 1
    }
}

println("All files completed")
println("Succesfully decrypted " + succesful)
println("Failed to decrypt " + failed)

Error: type mismatch;
 found   : List[scala.collection.immutable.IndexedSeq[String]]
 required: Null (2995)Error: type mismatch;
 found   : List[String]
 required: Null (3393)